## Метод Гаусса с выбором главного элемента
В методе Гаусса используется деление на диагональный элемент. Возникает вопрос: что будет, если диагональный элемент равен нулю или близок к нему? В этом случае метод Гаусса не даёт устойчивого решения. Более того, метод Гаусса устойчив только для матриц с диагональным преобладанием. Если диагонального преобладания нет, то можно использовать модификации метода Гаусса, которые считаются условно устойчивыми:

1. С выбором главного элемента в строке
2. С выбором главного элемента в столбце
3. С выбором главного элемента по всей матрице

Рассмотрим первый метод - с выбором главного элемента в строке. Вспомним, как начинался оригинальный метод: нужно было первую строку, умноженную на коэффициент вычесть из всех остальных. В модифицированном методе перед тем, как это делать, нужно найти в первой строке максимальный по модулю элемент и поменять местами первый столбец и столбец, содержащий максимальный элемент. Потом то же самое нужно сделать, когда будете вычитать вторую строку (только местами нужно будет менять второй столбец и столбец с максимальным элементом) и т.д.

Для примера пройдём прямой ход для модифицированной СЛАУ из предыдущего урока:
$$
\left\{\begin{array}{ccc}2x + y - z &=& 8 \\
-3x - y + 4z &=& -11 \\
-2x + y + 2z &=& -3 \end{array}\right.
$$

In [ ]:
# Задаём матрицы
A = Float64.([2 1 -1;
              -3 -1 4;
              -2 1 2]);
B = Float64.([8, -11, -3]);
A_pr = [A B]

Для поиска максимального элемента в массиве можно использовать функцию `argmax(A)`. Она находит индекс максимального элемента в коллекции.

In [ ]:
q = [1, 2, -5, 10, 4, 2];
argmax(q)

Найдём индекс максимального элемента по модулю в первой строке (учтите, нужно брать неприсоединённую матрицу). Для нахождения модуля числа используется функция `abs`.

In [ ]:
i = argmax(abs.(A_pr[1, 1:3]))

Видно, что первый элемент самый большой, менять ничего не нужно. Поэтому делаем ту же самую операцию, что и в предыдущий раз: вычитаем первую строку из второй и из третьей.

In [ ]:
A_pr[2, :] -= A_pr[1, :] * A_pr[2,1] / A_pr[1,1];
A_pr[3, :] -= A_pr[1, :] * A_pr[3,1] / A_pr[1,1];
A_pr

Снова ищем максимальный по модулю элемент, на этот раз во второй строке.

In [ ]:
i = argmax(abs.(A_pr[2, 2:3]))

Номер максимального элемента второй, но мы начинали со второго столбца, поэтому максимальный по модулю элемент находится в третьем столбце. Меняем местами третий и второй столбцы (номер столбца с максимальным элементом равен i+1). Обратите внимание на приём, который используется для того, чтобы поменять местами столбцы: их объединяют в кортеж.

In [ ]:
A_pr[:, 3], A_pr[:, 2] = A_pr[:, 2], A_pr[:, 3];
A_pr

Осталось вычесть вторую строку из третьей.

In [ ]:
A_pr[3, :] -= A_pr[2, :] * A_pr[3,2] / A_pr[2,2];
A_pr

### Упражнение 1
Напишите функцию `gauss_main_row(A::AbstractMatrix, B::AbstractVector)::AbstractVector`, которая принимает матрицу A и вектор B и возвращает вектор неизвестных с помощью метода Гаусса с выбором главного элемента по строке. 

In [ ]:
function gauss_main_row(A::AbstractMatrix, B::AbstractVector)::Vector{Float64}
    n = size(A,1)
    X = zeros(Float64, n) # задаём вектор X из нулей
    ord_X = collect(1:n)
    
    A_pr = [A B] # делаем присоединённую матрицу
    
    for i in 1:n-1          # главный цикл - осуществляет операцию для каждой строки
        k = argmax(abs.(A_pr[i, i:end-1]))
        if i!=k
            A_pr[:, k+i-1], A_pr[:, i] = A_pr[:, i], A_pr[:, k+i-1]
            ord_X[i], ord_X[k+i-1] = ord_X[k+i-1], ord_X[i]
        end
        for j in i+1:n      # вложенный цикл для вычитания
            A_pr[j, :] .-= A_pr[i, :] / A_pr[i, i] * A_pr[j, i] # из j-й строки вычесть i-ю, умноженную на соответствующие коэффициенты. Чтобы сделать это со всеми столбацами, используется срез.
        end
    end
    A_new = A_pr[1:end, 1:end-1] # Разделяем полученную матрицу
    B_new = A_pr[1:end, end]
    
    X[n] = B_new[n]/A_new[n, n] # находим последний X
    for i in n-1:-1:1 
        X[i] = (B_new[i] - sum(A_new[i, i+1:n] .* X[i+1:n]))/A_new[i,i] # последовательно, начиная с конца, находим X[i]. Используем значение X, полученное на предыдущем шаге
    end
    
    X_new = zeros(Float64, n)
    X_new[ord_X] .= X
    return X_new
end

In [ ]:
A = Float64.([2 1 -1;
              -3 -1 2;
              -2 1 2]);

B = Float64.([8, -11, -3])
X = gauss_main_row(A, B)
@assert X ≈ A \ B

### Упражнение 2
По аналогии с предыдущим, создайте функцию `gauss_main_col(A::AbstractMatrix, B::AbstractVector)::AbstractVector`, которая принимает матрицу A и вектор B и возвращает вектор неизвестных с помощью метода Гаусса с выбором главного элемента по столбцу. Она должна отличаться парой строк в цикле прямого хода (искать главный элемент нужно по столбцам и менять местами не столбцы, а строки). Также, когда меняются местами строки в матрицах A и B, вектор X не меняется.

In [ ]:
function gauss_main_col(A::AbstractMatrix, B::AbstractVector)::Vector{Float64}
    n = size(A,1)
    X = zeros(Float64, n) # задаём вектор X из нулей
    
    A_pr = [A B] # делаем присоединённую матрицу
    
    for i in 1:n-1          # главный цикл - осуществляет операцию для каждой строки
        k = argmax(abs.(A_pr[i:end, i]))

        if i!=k
            A_pr[k+i-1, :], A_pr[i, :] = A_pr[i, :], A_pr[k+i-1, :]
        end
        for j in i+1:n      # вложенный цикл для вычитания
            A_pr[j, :] .-= A_pr[i, :] / A_pr[i, i] * A_pr[j, i] # из j-й строки вычесть i-ю, умноженную на соответствующие коэффициенты. Чтобы сделать это со всеми столбацами, используется срез.
        end
    end
    A_new = A_pr[1:end, 1:end-1] # Разделяем полученную матрицу
    B_new = A_pr[1:end, end]
    
    X[n] = B_new[n]/A_new[n, n] # находим последний X
    for i in n-1:-1:1 
        X[i] = (B_new[i] - sum(A_new[i, i+1:n] .* X[i+1:n]))/A_new[i,i] # последовательно, начиная с конца, находим X[i]. Используем значение X, полученное на предыдущем шаге
    end
    
    return X
end

In [ ]:
A = Float64.([2 1 -1;
              -3 -1 2;
              -2 1 2]);

B = Float64.([8, -11, -3])
X = gauss_main_col(A, B)
@assert X ≈ A \ B